In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os

from utils.data_process import *
from utils.eval import *
from utils.aggregate import *
os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
df_gt = pd.read_csv('./datasets/Medical/GroundTruth.csv')
df_worker = pd.read_csv('./datasets/Medical/WorkerData.csv')
df_gt['bbox'] = df_gt['bbox'].apply(lambda x: [int(i) for i in re.findall(r'\d+', x)])

df_worker['worker_bbox'] = df_worker['worker_bbox'].apply(convert_to_list)
df_worker['sam_bbox'] = df_worker['sam_bbox'].apply(convert_to_list)


df_worker['worker_area'] = df_worker['worker_bbox'].apply(lambda x: (x[2]-x[0])*(x[3]-x[1]))
df_worker['sam_area'] = df_worker['sam_bbox'].apply(lambda x: (x[2]-x[0])*(x[3]-x[1]))
df_worker[(df_worker['sam_area'] < 400) & (df_worker['sam_conf'] < 0.7)]
df_worker = df_worker[(df_worker['sam_area'] >= 2924) | (df_worker['sam_conf'] >= 0.7)]
fill_weight_df(df_worker)


,worker_bbox,author,author_id,img_name,weight,cluster,category,sam_conf,sam_bbox,worker_area,sam_area
0,"[6.0, 1272.0, 407.0, 1777.0]",梁春霞,7,%5B39450_53003%5D%5B39519_53078%5D%5B2%5D%5B0....,0.820932,NaN,其他正常细胞,0.979302,"[0.0, 1320.0, 424.0, 1759.0]",202505.0,186136.0
1,"[1393.0, 596.0, 2187.0, 1236.0]",赵辉,4,%5B39450_53003%5D%5B39519_53078%5D%5B2%5D%5B0....,0.532935,NaN,其他正常细胞,0.888255,"[1317.0, 650.0, 2038.0, 1119.0]",508160.0,338149.0
2,"[15.0, 1277.0, 489.0, 1844.0]",禾荣华,3,%5B39450_53003%5D%5B39519_53078%5D%5B2%5D%5B0....,0.645175,NaN,其他正常细胞,0.952214,"[0.0, 1320.0, 424.0, 1759.0]",268758.0,186136.0
3,"[987.0, 424.0, 1263.0, 661.0]",禾荣华,3,%5B39450_53003%5D%5B39519_53078%5D%5B2%5D%5B0....,0.411228,NaN,其他正常细胞,0.877491,"[1040.0, 475.0, 1224.0, 706.0]",65412.0,42504.0
4,"[0.0, 1248.0, 413.0, 1842.0]",贺代碧,5,%5B39450_53003%5D%5B39519_53078%5D%5B2%5D%5B0....,0.718020,NaN,其他正常细胞,0.977822,"[0.0, 1320.0, 424.0, 1759.0]",245322.0,186136.0
...,...,...,...,...,...,...,...,...,...,...,...
14574,"[182.0, 1228.0, 486.0, 1605.0]",赵辉,4,%5B28764_20425%5D%5B28937_20632%5D%5B2%5D%5B0....,0.580064,NaN,其他正常细胞,0.944339,"[195.0, 1236.0, 435.0, 1513.0]",114608.0,66480.0
14575,"[1167.0, 1212.0, 1617.0, 1556.0]",赵辉,4,%5B28764_20425%5D%5B28937_20632%5D%5B2%5D%5B0....,0.520640,NaN,其他正常细胞,0.958145,"[1201.0, 1271.0, 1606.0, 1470.0]",154800.0,80595.0
14576,"[1722.0, 782.0, 2176.0, 1414.0]",赵辉,4,%5B28764_20425%5D%5B28937_20632%5D%5B2%5D%5B0....,0.626985,NaN,化生细胞,0.820144,"[1697.0, 854.0, 2221.0, 1320.0]",286928.0,244184.0
14577,"[3540.0, 1560.0, 3807.0, 1892.0]",赵辉,4,%5B28764_20425%5D%5B28937_20632%5D%5B2%5D%5B0....,0.478481,NaN,化生细胞,0.840806,"[3617.0, 1531.0, 3839.0, 1950.0]",88644.0,93018.0


In [3]:

threshold = 0.20

img2df = {}
total_df_removed = pd.DataFrame(columns=['worker_bbox', 'author', 'author_id', 'img_name', 'weight', 'cluster', 'category', 'sam_conf', 'sam_bbox', 'worker_area', 'sam_area', 'remove'])

author2giou = get_author2giou(df_worker)
author2quality = {author: np.mean(gious) for author, gious in author2giou.items()}


In [4]:

img2_cluster = {}
for img_name in tqdm(df_worker['img_name'].unique()[:]):
    temp_df = df_worker[df_worker['img_name'] == img_name]
    temp_df.reset_index(drop=True, inplace=True)
    temp_df_cluster,df_removed = iter_cluster(temp_df,author2quality,threshold = threshold,\
        max_iter = 1,if_remove_outlier = True)
    
    if len(total_df_removed) == 0:
        total_df_removed = df_removed
    else:
        total_df_removed = pd.concat([total_df_removed, df_removed], ignore_index=False)

    try:
        temp_df_cluster = fill_weight_df(temp_df_cluster)
    except Exception as e:
        print(e)
        print(temp_df_cluster)
        break

    author2giou = get_author2giou(temp_df_cluster)
    temp_df_aggregated = aggregate_for_all_images(temp_df_cluster, author2giou, total_df_removed, method = 'mean', weight = 'all')
    temp_df_aggregated['giou'] = temp_df_aggregated.apply(lambda x: calculate_giou(x['worker_bbox'], x['sam_bbox']), axis=1)

    fill_weight_df(temp_df_aggregated)

    local_weight = np.mean(temp_df_aggregated['giou'])
    sam_weight = np.mean(temp_df_aggregated['sam_conf'])
    weight = (local_weight + sam_weight) / 2

    temp_df_aggregated['final_box'] = temp_df_aggregated.apply(lambda x: [np.average(k, weights=[weight, 1-weight]) for k in zip(x['sam_bbox'], x['worker_bbox'])], axis=1)
    
    img2df[img_name] = temp_df_aggregated
    img2_cluster[img_name] = temp_df_cluster


print("threshold",threshold)
print("len(total_df_removed)",len(total_df_removed))

braylan_f1_scores, one2one_f1_scores, acc = temp_eva(img2df,df_gt)
print("braylan_f1_scores",np.mean(braylan_f1_scores))
print("one2one_f1_scores",np.mean(one2one_f1_scores))
print("acc",acc)







100%|██████████| 446/446 [02:31<00:00,  2.95it/s]


threshold 0.2
len(total_df_removed) 740
braylan_f1_scores 0.8251646703922692
one2one_f1_scores 0.8205310751372944
acc 0.9160828273392496
